#  Hacks: MovieChat Movie Database.
> Using python to create a database for my movie review website
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

In [12]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

app = Flask(__name__)
database = 'sqlite:///sqlite.db' 
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for movies table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class Movie
    - db.Model inheritance
    - _init_ method
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods

In [13]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

class Movie(db.Model):
    __tablename__ = 'movies'  

    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _reason = db.Column(db.String(255), unique=False, nullable=False)

    def __init__(self, name, uid, reason, password="123qwerty",):
        self._name = name   
        self._uid = uid
        self.set_password(password)
        self._reason = reason

    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        self._name = name
    
    @property
    def uid(self):
        return self._uid
    
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." 

    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    @property
    def reason(self):
        return self._reason
    
    @reason.setter
    def reason(self, reason):
        self._reason = reason
        
    def is_reason(self, reason):
        return self._reason == reason
    
    def __str__(self):
        return json.dumps(self.read())

    def create(self):
        try:
            db.session.add(self) 
            db.session.commit()  
            return self
        except IntegrityError:
            db.session.remove()
            return None

    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "reason": self.reason,
        }

    def update(self, name="", uid="", password="", reason=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        if len(reason) > 0:
            self.name = reason
        db.session.add(self)
        db.session.commit()
        return self

    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. Movie Object Constructors
    3. Try / Except 


In [14]:
"""Database Creation and Testing """


def initMovies():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = Movie(name='Interstellar', uid='toby', password='123toby', reason='Beautiful graphics')
        u2 = Movie(name='Click', uid='niko', password='123niko', reason='Adam Sandler funny')

# Example users with movie reviews

        movies = [u1, u2,]

        """Builds sample movie/note(s) data"""
        for movie in movies:
            try:
                '''add movie to table'''
                object = movie.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {movie.uid}, or error.")
                
initMovies()

Records exist uid toby, or error.
Records exist uid niko, or error.


## Check for given Credentials in movies table in sqlite.db
> Use of ORM Query object and custom methods to identify movie to credentials uid and password

- Comment on purpose of following
    1. Movie.query.filter_by
    2. movie.password

In [15]:
def find_by_uid(uid):
    with app.app_context():
        movie = Movie.query.filter_by(_uid=uid).first()
    return movie # returns movie object

def check_credentials(uid, password):
    movie = find_by_uid(uid)
    if movie == None:
        return False
    if (movie.is_password(password)):
        return True
    return False
        


## Create a new Movie Review in Sqlite.db

In [16]:
def create():
    uid = input("Enter your user id:")
    movie = find_by_uid(uid)
    try:
        print("Found\n", movie.read())
        return
    except:
        pass # keep going
    
    name = input("Enter your film's name:")
    password = input("Enter your password")
    reason = input("Enter the reason you enjoyed the film")
    
    movie = Movie(name=name, 
                uid=uid, 
                password=password,
                reason=reason
                )
           
    with app.app_context():
        try:
            object = movie.create()
            print("Created\n", object.read())
        except: 
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 5, 'name': 'Back to the Future', 'uid': 'adin', 'reason': 'Kinda cool'}


## Reading movies table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. Movie.query.all
    2. json_ready assignment, google List Comprehension

In [17]:

# SQLAlchemy extracts all movies from database, turns each movie into JSON
def read():
    with app.app_context():
        table = Movie.query.all()
    json_ready = [movie.read() for movie in table] # "List Comprehensions", for each movie add movie.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Interstellar',
  'uid': 'toby',
  'reason': 'Beautiful graphics'},
 {'id': 2, 'name': 'Click', 'uid': 'niko', 'reason': 'Adam Sandler funny'},
 {'id': 3,
  'name': 'Puss and Boots the Last Wish',
  'uid': 'Luke',
  'reason': 'the animation'},
 {'id': 4, 'name': 'Over The Hedge', 'uid': 'john', 'reason': 'Fun Comedy'},
 {'id': 5,
  'name': 'Back to the Future',
  'uid': 'adin',
  'reason': 'Kinda cool'}]

# Update and Delete Functions

In [10]:
def update():
    uid = input("Enter your user id:")
    movie = find_by_uid(uid)
    try:
        print("Found\n", movie.read())
        pass
    except:
        return
    
    password = input("Enter your password:")
    if check_credentials(uid, password):
        pass
    else:
        return
    
    nname = input("Enter your name")
    nuid = input("Enter your new user id")
    try:
        print("Already exists\n", find_by_uid(nuid).read())
        return
    except:
        pass

    with app.app_context():
        try:
            object = movie.update(nname, nuid, "")
            print("updated\n", object.read())
        except:
            print("Unknown error")
    
        
update()

In [46]:
def delete():
    uid = input("Enter your user id:")
    movie = find_by_uid(uid)
    try:
        print("Found\n", movie.read())
        pass
    except:
        return
    
    password = input("Enter your password:")
    if check_credentials(uid, password):
        pass
    else:
        return

    with app.app_context():
        try:
            object = movie.delete()
            print("deleted\n", object.read())
        except:
            print("Unknown error")
    
        
delete()

Found
 {'id': 7, 'name': 'Luke', 'uid': 'Luke', 'dob': '03-15-2023', 'age': 0}


In [11]:
def crud():
    uinp = input("create, read, update, or delete?")
    if uinp.lower() == "create":
        create()
    elif uinp.lower() == "read":
        read()
    elif uinp.lower() == "update":
        update()
    elif uinp.lower() == "delete":
        delete()
    else:
        print("Input Error")
crud()

Created
 {'id': 4, 'name': 'Over The Hedge', 'uid': 'john', 'reason': 'Fun Comedy'}
